<a href="https://colab.research.google.com/github/gmauricio-toledo/tda/blob/main/notebooks/01-PCA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<h1>PCA</h1>

En esta notebook analizaremos varios aspectos teóricos y prácticos del método PCA. En particular, mostraremos ejemplos de cómo podemos perder información topológica del conjunto de datos al hacer reducción de dimensionalidad.

# ¿Cómo usar PCA?

Un conjunto de datos:

In [ ]:
from sklearn.datasets import make_classification
import numpy as np
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA

puntos, _ = make_classification(n_samples=150,
                                n_features=2,
                                n_classes=1,
                                n_informative=2,
                                n_redundant=0,
                                n_clusters_per_class=1,
                                scale=[0.5,2],
                                random_state=721
                                )

media = np.mean(puntos, axis=0)  # ¿QUÉ ESTAMOS HACIENDO?

#----------- GRAFICAR -----------
plt.figure(figsize=(3,3))
plt.scatter(puntos[:,0], puntos[:,1],color='gray',s=20)
plt.scatter([media[0]],[media[1]],color='red',marker='x', label='Mean') # (?)
plt.axhline(y=0, color='k', alpha=0.25)
plt.axvline(x=0, color='k', alpha=0.25)
plt.axis('equal')
plt.legend(loc='best')
plt.show()

Uso de PCA:

In [ ]:
# Inicializamos y entrenamos PCA
pca = PCA(n_components=2)  # Inicialización (instanciación)
puntos_pca = pca.fit_transform(puntos)  # Entrenamiento y Transformación

# O por separado:
# pca.fit(puntos)  # Entrenamiento
# puntos_pca = pca.transform(puntos)  # Transformación

Observar como podemos consultar información relevante sobre el procedimiento en la clase entrenada.

In [ ]:
# Consultamos información sobre el método
pcs = pca.components_   # Vectores componentes principales (PC)
mean = pca.mean_        # Promedio de los puntos

pcs.shape, mean.shape

In [ ]:
# Graficamos la información
plt.figure(figsize=(6,3))
plt.subplot(1,2,1) # Grafica con los datos antes de la transformación
plt.scatter(puntos[:,0], puntos[:,1],color='gray',s=20) # Graficamos cada una de las PC
for i in range(2): # Graficamos cada una de las dos componentes principales
    v = pcs[i]
    plt.arrow(mean[0], mean[1], v[0], v[1], head_width=0.05, head_length=0.1, fc='k', ec='k')
plt.scatter([mean[0]],[mean[1]],color='red',marker='x') # Graficamos el promedio
plt.axhline(y=0, color='k', alpha=0.25) # El eje X
plt.axvline(x=0, color='k', alpha=0.25) # El eje Y
plt.axis('equal')
plt.subplot(1,2,2) # Gráfica con los datos después de la transformación
plt.scatter(puntos_pca[:,0], puntos_pca[:,1],color='gray',s=20)
plt.axhline(y=0, color='k', alpha=0.25)
plt.axvline(x=0, color='k', alpha=0.25)
plt.axis('equal')
plt.show()

🟡 **PENDIENTE**: Veamos que se cumple la condición que impusimos sobre las coordenadas de las proyecciones

$$\sum_{j=1}^d \beta_j = \sum_{j=1}^d V^T(x_j - \mu(A) ) = 0$$

In [ ]:
np.sum(puntos_pca,axis=0)

Las componentes principales forman una base ortonormal

In [ ]:
np.dot(pcs.T,pcs)

Varianza explicada de los datos originales

In [ ]:
pca.explained_variance_ratio_

🔵 ¿Qué podríamos concluir de la forma de los datos con esta información?

## 🟡 Inclusión de dimensiones inferiores

Supongamos que queremos proyectar un conjunto $A\subset\mathbb{R}^D$ en espacios $d_1,d_2$ dimensionales con $d_2< d_1< D$.

En lugar de entrenar dos modelos de PCA para $d_1$ y $d_2$ respectivamente, basta con entrenar un modelo para $d_1$ para obtener la reducción $A_{d_1}$. Para encontrar la proyección a $\mathbb{R}^{d_2}$ basta con tomar las $d_2$ primeras columas de la representación matricial de $A_{d_2}$.

A continuación ilustramos experimentalmente esto

In [ ]:
from sklearn.datasets import make_classification
import numpy as np
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA


puntos, _ = make_classification(n_samples=500,
                                n_features=20,
                                n_classes=3,
                                n_informative=15,
                                n_clusters_per_class=2,
                                random_state=721
                                )

puntos.shape

pca_15 = PCA(n_components=15)
puntos_pca_15 = pca_15.fit_transform(puntos)
cum_sum_15_rounded = np.round(np.cumsum(pca_15.explained_variance_ratio_),3)

pca_8 = PCA(n_components=8)
puntos_pca_8 = pca_8.fit_transform(puntos)
cum_sum_8_rounded = np.round(np.cumsum(pca_8.explained_variance_ratio_),3)

#---- Comparación de las varianzas ---
comparacion = np.isclose(puntos_pca_8, puntos_pca_15[:,:8]).all()
print(f"Son iguales las primeras 8 columas de la reducción 15-dimensional: {comparacion}")
print(f"Varianza explicada acumulada de la reducción 15-dimensional:\n{cum_sum_15_rounded}")
print(f"Varianza explicada acumulada de la reducción 8-dimensional:\n{cum_sum_8_rounded}")

Tomemos un punto aleatorio y comparemos sus coordenadas

In [ ]:
idx = np.random.choice(puntos.shape[0],size=1)
punto_15d = puntos_pca_15[idx]
punto_8d = puntos_pca_8[idx]
print(f"Coordenadas en 15-dimensional:\n{punto_15d}")
print(f"Coordenadas en 8-dimensional:\n{punto_8d}")

In [ ]:
plt.figure(figsize=(6,3))
plt.subplot(1,2,1)
plt.scatter(puntos_pca_15[:,0], puntos_pca_15[:,1],color='gray')
plt.title('Primeras dos componentes\nPCA 15D')
plt.subplot(1,2,2)
plt.scatter(puntos_pca_8[:,0], puntos_pca_8[:,1],color='gray')
plt.title('Primeras dos componentes\nPCA 8D')
plt.tight_layout()
plt.show()

## Ejemplos

Ahora veamos varios ejemplos ilustrativos de la reducción de dimensionalidad con PCA:

* Datos lineales/no lineales con/sin patrones

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA

xs = np.random.uniform(size=(100,), low=-1,high=3)

#------- un patrón en los datos -------------
# xs1 = np.random.uniform(size=50, low=-1,high=0)
# xs2 = np.random.uniform(size=50, low=2,high=3)
# xs = np.hstack((xs1,xs2))

ys = -1.5*xs + 2
# ys = -1.5*xs + 2 + np.random.normal(size=(100,), scale=0.25)  # Es útil para quitar el ruido
# ys = -(xs-2)**3 - 0.5*(ys+2)**2  # No funciona bien en datos que no tienen una relación lineal

puntos = np.array([xs,ys]).T

pca = PCA(n_components=2)
pca.fit(puntos)
puntos_pca = pca.transform(puntos)

pcs = pca.components_
mean = pca.mean_

fig, axs = plt.subplots(1, 2, figsize=(12, 4))
axs[0].scatter(xs,ys,color='red')
axs[0].axhline(y=0, color='k', alpha=0.5)
axs[0].axvline(x=0, color='k', alpha=0.5)
for i in range(2):
    v = pcs[i]
    axs[0].arrow(mean[0], mean[1], v[0], v[1], head_width=0.05, head_length=0.1, fc='k', ec='k')
axs[0].set_title('Puntos originales')
axs[0].axis('equal')
axs[1].scatter(puntos_pca[:,0],np.zeros(puntos_pca.shape[0]),color='red')
axs[1].axhline(y=0, color='k', alpha=0.5)
axs[1].axvline(x=0, color='k', alpha=0.5)
axs[1].set_title('Puntos proyectados')
axs[1].axis('equal')
fig.tight_layout()
fig.show()

Un ejemplo con datos en $\mathbb{R}^3$

In [ ]:
#@title Función para graficar en HTML puntos en $\mathbb{R}^3$

import numpy as np
import plotly.graph_objects as go
import plotly.offline as offline


def scatter_plot_3d_plotly(X,y=None,filename='plot3d.html',fig_title='Plot'):
    assert X.shape[1] == 3, "X debe tener 3 dimensiones"
    if y is not None:
        assert X.shape[0] == y.shape[0], "X y y deben tener la misma cantidad de puntos"
    else:
        y = np.zeros(X.shape[0])
    N = X.shape[0]
    # Extraer coordenadas x, y, z
    x = X[:, 0]
    y = X[:, 1]
    z = X[:, 2]

    # Crear la figura 3D
    fig = go.Figure(data=[go.Scatter3d(
        x=x,
        y=y,
        z=z,
        mode='markers',
        marker=dict(
            size=3,
            color=y,  # Colorear por valor de clase
            colorscale='Viridis',  # Escala de colores
            opacity=0.8
        ),
        text=[f'{y[i]}' for i in range(N)],  # Texto al hacer hover (opcional)
        hovertemplate='<b>%{text}</b><extra></extra>'
    )])

    # Configurar el layout para elementos de interfaz
    fig.update_layout(
        title=fig_title,
        scene=dict(
            # Ocultar ejes coordenados
            xaxis=dict(
                visible=False,
                showbackground=False,
                showgrid=False,
                zeroline=False,
            ),
            yaxis=dict(
                visible=False,
                showbackground=False,
                showgrid=False,
                zeroline=False,
            ),
            zaxis=dict(
                visible=False,
                showbackground=False,
                showgrid=False,
                zeroline=False,
            ),
            # Configurar cámara y aspecto
            camera=dict(
                up=dict(x=0, y=0, z=1),
                center=dict(x=0, y=0, z=0),
                eye=dict(x=1.2, y=1.2, z=1.2)
            )
        ),
        # Ocultar elementos de la interfaz
        showlegend=False,
        margin=dict(l=0, r=0, b=0, t=50),
    )
    # Guardar como archivo HTML
    offline.plot(fig, filename=filename, auto_open=False)

In [ ]:
import numpy as np

xs = np.random.uniform(size=(200,), low=-1,high=3)
ys = np.random.uniform(size=(200,), low=2,high=4)

zs = -1.5*xs + 2*xs + np.random.normal(size=(200,), scale=0.1)
# zs = 1.5*xs**2 - 2*xs**2 + np.random.normal(size=(200,), scale=0.1)

puntos = np.array([xs,ys,zs]).T # Unimos los puntos

scatter_plot_3d_plotly(puntos,filename='pca3d-example-3.html',fig_title='Plot') # Usamos la función para graficar en HTML

# PCA 2D y gráfica
pca = PCA(n_components=2)
pca.fit(puntos)
puntos_pca = pca.transform(puntos)

plt.figure(figsize=(3,3))
plt.scatter(puntos_pca[:,0], puntos_pca[:,1],color='gray',s=20)
plt.axhline(y=0, color='k', alpha=0.25)
plt.axvline(x=0, color='k', alpha=0.25)
plt.axis('equal')
plt.show()

## Perdida de propiedades topológicas con PCA

Si PCA preservara propiedades topológicas no habría mucha necesidad de desarrollar herramientas del análisis topológico de datos

In [ ]:
from scipy.stats import ortho_group
import numpy as np
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE
import os

def encajar(X,dim_output=50):
    n, dim_input = X.shape
    dataset = np.zeros((n, dim_output))
    dataset[:,:dim_input] = X
    # ROTAR CON UNA MATRIZ ALEATORIA Y LUEGO TRASLADAR ALEATORIAMENTE
    Q = ortho_group.rvs(dim_output)
    dataset = dataset @ Q.T
    # traslacion = np.random.randn(dim_output)
    # dataset = dataset + traslacion
    return dataset

🔵 Encuentra un conjunto de datos que tenga un cíclo de dimensión 1 en alta dimensión tal que se pierda este ciclo al proyectar a 2D haciendo PCA

In [ ]:
ts = np.random.uniform(size=(500,), low=0,high=2*np.pi)
xs = np.sin(ts)
ys = 0.3*np.cos(ts)
zs = np.random.uniform(size=(500,), low=-1,high=1)

plt.figure()
plt.scatter(xs,ys,color='gray',s=20)
plt.axis('equal')
plt.show()

dim = 300

X = np.zeros((500,3))
X[:,0] = xs
X[:,1] = ys
X[:,2] = zs

scatter_plot_3d_plotly(X,filename='cilindro_ruido_3d.html',fig_title='Plot')

X = encajar(X,dim_output=300)

pca = PCA(n_components=2)
pca.fit(X)
X_pca = pca.transform(X)

plt.figure()
plt.scatter(X_pca[:,0],X_pca[:,1])
plt.show()

In [ ]:
pca = PCA(n_components=3)
pca.fit(X)
X_pca = pca.transform(X)
scatter_plot_3d_plotly(X_pca,filename='cilindro_ruido_PCA3d.html',fig_title='Plot')

# 🟡 Efecto de PCA en una tarea de regresión

Veamos el efecto de la reducción de dimensionalidad en una tarea de regresión

El dataset

In [ ]:
import pandas as pd

train_path = '/content/sample_data/california_housing_train.csv'
test_path = '/content/sample_data/california_housing_test.csv'

train_df = pd.read_csv(train_path)
test_df = pd.read_csv(test_path)
train_df

Separamos en $X$, $y$

In [ ]:
X_train = train_df.drop(columns=['median_house_value']).values
X_test = test_df.drop(columns=['median_house_value']).values
y_train = train_df['median_house_value'].values
y_test = test_df['median_house_value'].values

print(X_train.shape, y_train.shape)
print(X_test.shape, y_test.shape)

## PCA 2d

In [ ]:
from sklearn.decomposition import PCA

pca = PCA(n_components=2)
X_train_pca = pca.fit_transform(X_train)
X_test_pca = pca.transform(X_test)

In [ ]:
import matplotlib.pyplot as plt

plt.figure()
plt.scatter(X_train_pca[:,0],X_train_pca[:,1],color='red',label='train')
plt.scatter(X_test_pca[:,0],X_test_pca[:,1],color='blue',label='test')
plt.legend()
plt.show()

🔵 Observar la distribución de datos de entranamiento y prueba

In [ ]:
plt.figure()
plt.scatter(X_train_pca[:,0],X_train_pca[:,1],c=y_train)
plt.colorbar()
plt.show()

## Comparación regresión con el dataset original y PCA

In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error

lr = LinearRegression()
lr.fit(X_train,y_train)
y_train_pred = lr.predict(X_train)
y_test_pred = lr.predict(X_test)

train_mae = mean_absolute_error(y_train,y_train_pred)
test_mae = mean_absolute_error(y_test,y_test_pred)

print(f"Train MAE: {train_mae}")
print(f"Test MAE: {test_mae}")

In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error

lr = LinearRegression()
lr.fit(X_train_pca,y_train)
y_train_pred = lr.predict(X_train_pca)
y_test_pred = lr.predict(X_test_pca)

train_mae = mean_absolute_error(y_train,y_train_pred)
test_mae = mean_absolute_error(y_test,y_test_pred)

print(f"Train MAE: {train_mae}")
print(f"Test MAE: {test_mae}")

In [ ]:
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt

pca_full = PCA(n_components=8)
pca_full.fit(X_train)
print(pca_full.explained_variance_ratio_)

plt.figure()
plt.plot(pca_full.explained_variance_ratio_)
plt.xlabel('Number of Components')
plt.ylabel('Explained Variance Ratio')
plt.ylim(0,1)
plt.show()

Exploremos el efecto del número del número de componentes en el error de las predicciones

In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error

X_train_pca_full = pca_full.transform(X_train)
X_test_pca_full = pca_full.transform(X_test)

train_maes = []
test_maes = []

for k in range(1,9):
    X_train_pca_k = X_train_pca_full[:,:k]
    X_test_pca_k = X_test_pca_full[:,:k]
    lr = LinearRegression()
    lr.fit(X_train_pca_k,y_train)
    y_train_pred = lr.predict(X_train_pca_k)
    y_test_pred = lr.predict(X_test_pca_k)
    train_mae = mean_absolute_error(y_train,y_train_pred)
    test_mae = mean_absolute_error(y_test,y_test_pred)
    train_maes.append(train_mae)
    test_maes.append(test_mae)

plt.figure()
plt.plot(train_maes,label='train')
plt.plot(test_maes,label='test')
plt.legend()
plt.xlabel('Number of Components')
plt.ylabel('MAE')
plt.show()

## Comparación de regresión con el dataset original y selección de variables

Ahora intentemos probar con selección de variables, es decir, proyecciones canónicas en los ejes

In [ ]:
from sklearn.feature_selection import SelectKBest, f_regression
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error

train_maes = []
test_maes = []

for k in range(1,9):
    # Selección de variables
    selector = SelectKBest(f_regression, k=k)
    X_train_selected = selector.fit_transform(X_train, y_train)
    X_test_selected = selector.transform(X_test)
    # Entrenamiento del modelo
    lr = LinearRegression()
    lr.fit(X_train_selected,y_train)
    # Predicción
    y_train_pred = lr.predict(X_train_selected)
    y_test_pred = lr.predict(X_test_selected)
    # Evaluación
    train_mae = mean_absolute_error(y_train,y_train_pred)
    test_mae = mean_absolute_error(y_test,y_test_pred)
    train_maes.append(train_mae)
    test_maes.append(test_mae)

plt.figure()
plt.plot(train_maes,label='train')
plt.plot(test_maes,label='test')
plt.legend()
plt.xlabel('Number of Components')
plt.ylabel('MAE')
plt.show()